In [88]:
#Knižnice
import pandas as pd
import matplotlib.pyplot as plt
# Uporabljal sem predvsem knižnjico plotly, ki omogoča širok nabor grafičnih možnosti ter je interaktivna.
import plotly.express as px
import plotly.graph_objects as go
import pycountry_convert as pc #To je knižnjica za pretvorbo držav v kontinente

#Nalaganje podatkov
podjetja1 = pd.read_csv("urejeni_podatki/lestvica_podjetij.csv")
podjetja2 = pd.read_csv("urejeni_podatki/lestvica_podjetij_2.csv")

In [96]:
#Groba obdelava razpredelnic
podjetja = pd.merge(podjetja1, podjetja2, left_on = "ime", right_on="ime", how='inner')
podjetja['Donosnost na prodajo'] = podjetja.profit_B/podjetja.prodaja_B
podjetja['Donosnost na premozensko vrednost'] = podjetja.profit_B/podjetja.premozenska_vrednost_B

podjetja_po_drzavi = podjetja.groupby("drzava").agg({'ime' : ['count'], 'premozenska_vrednost_B' : ['sum']})
podjetja_po_panogi = podjetja.groupby("panoga").agg({'stevilo_zaposlenih':['mean','median'], 
                                                     "profit_B" : ['mean', 'median'], 
                                                     'leto_ustanovitve' :['median'], 
                                                      'Donosnost na prodajo' : ['mean'],
                                                       'Donosnost na premozensko vrednost' : ['mean'],
                                                       'trzna_cena_B' : ['median']})
podjetja_po_letu_ustanovitve = podjetja.groupby("leto_ustanovitve")

podjetja_po_panogi.columns = [x[0]+'_'+x[1] for x in podjetja_po_panogi.columns]
podjetja_po_drzavi.columns = [x[0]+'_'+x[1] for x in podjetja_po_drzavi.columns]

Za začetek si oglejmo kakšen vzorec podatkov imamo.

In [16]:

def drzava_v_kontinent(drzava):
    koda_drzave = pc.country_name_to_country_alpha2(drzava, cn_name_format="default")
    continent_name = pc.country_alpha2_to_continent_code(koda_drzave)
    return continent_name

podjetja['kontinent'] = podjetja.drzava.apply(lambda x: drzava_v_kontinent(x))

#Pretvorba imen kontinentov
kontinent_dict = {'EU':'Europa', 'NA' : 'Severna amerka', 'AS': 'Azija', 'OC' : 'Australija', 'SA' : 'Južna amerika'}
podjetja['kontinent'] = podjetja.kontinent.apply(lambda x: kontinent_dict[x])

df  = podjetja.groupby(["panoga", "kontinent"]).agg({'ime':'count'}).reset_index()

# Poizkus sortiranja
#V razpredelnici zadnja elementa iz meni neznanega razloga ne posortira pravilno.
sort = df.groupby(["panoga"]).agg({'ime':'sum'})
sort.columns= ['ime_sort']
sort = sort.reset_index()
df= pd.merge(df, sort, on="panoga", how='left').sort_values('ime_sort')

#Graf
graf1 = px.bar(df, x="panoga", y="ime",
             color='kontinent',
             height=600,
             width = 1300, 
             title = "Pregled podatkov",
             )
graf1.update_layout(
    title_font = {'size': 24},
    legend = {
        'title_text' : 'Kontinenti:',
        'font' : {'size': 13},
        'title_font' : {'size': 15}
    }
)
graf1.update_xaxes(
        title_text = 'Panoga',
        title_standoff = 20,
        tickangle = 45
)
graf1.update_yaxes(
    title_text = "Število podjetij"
)
graf1.show()

In [4]:
podjetja_po_drzavi=podjetja_po_drzavi.sort_values('ime_count', ascending=False)
graf5 = go.Figure(data=[
    go.Bar(name='Število podjetij',
     x = podjetja_po_drzavi.index, y = podjetja_po_drzavi.ime_count, yaxis='y', offsetgroup=1),
    go.Bar(name='Vsota tržnih vrednosti',
     x = podjetja_po_drzavi.index, y = podjetja_po_drzavi.premozenska_vrednost_B_sum, yaxis='y2', offsetgroup=2)
], layout={
    'yaxis' : {'title': 'Število podjetij'},
    'yaxis2' : {'title' : 'Premoženska vrednost[trilioni $]', 'overlaying': 'y', 'side': 'right'}
    
})
graf5.update_layout(
    width = 1200,
    height = 800,
    title = 'Moč držav',
    title_font = {'size': 24} ,
    legend = {
        'font' : {'size': 13},
    }
)
graf5.update_xaxes(
    title = 'Države'
)
graf5.update_layout(barmode='group')
graf5.show()

Kot pričakovano na lestvici kraljujeta ZDA in Kitajska. Pomemben je podatek, da ima večina držav zanemarljivo majhno število podjetij v lestvici. Sedaj si ogledamo razmerje med panogami in številom zaposlenih.

In [24]:
podjetja_po_panogi=podjetja_po_panogi.sort_values('stevilo_zaposlenih_mean', ascending=False)

graf3 = go.Figure(data=[
    go.Bar(name='povprečno število zaposlenih', x=podjetja_po_panogi.index, y=podjetja_po_panogi.stevilo_zaposlenih_mean/1000),
    go.Bar(name='mediana števila zaposlenih', x=podjetja_po_panogi.index, y=podjetja_po_panogi.stevilo_zaposlenih_median / 1000)
])
graf3.update_layout(
    title = "Panoge in število zaposlenih",
    title_font = {'size': 24},
    legend = {
        'font' : {'size': 13},
    },
    width = 1200,
    height = 600

)
graf3.update_xaxes(
        tickangle = 45,
        title_text = "Panoge",
        title_font = {"size": 12},
        title_standoff = 20)
graf3.update_yaxes(
    title_text = "število zaposlenih[tisoč]",
    title_font = {"size" : 12}
)

# Change the bar mode
graf3.update_layout(barmode='group')


graf3.show()

Podatki so pričakovani. Največje število delavcev imajo podjetja z prodajo in turizmom. Manjša števila pa pritičejo industrijskim obratom in bolj usmerjenim disciplinam. Spomnimo se, da prvi graf pokaže da imajo nekatere panoge zelo malo predstavnikov, npr. Hotels, Restaurants & Leisure le enega. 
Naslednja tama bo odvisnost mesta na lestvici od ostalih kvalitet. Zelo lepo korelacijo sta pokazala profit in tržna cena, kot kaže spodnji graf.

In [6]:
graf2 = px.scatter(podjetja, x="profit_B", y="trzna_cena_B",
             color='mesto')
graf2.update_layout(
    title = "Odvisnost trzne cene in profita od mesta na lestvici",
    title_font = {'size': 24},
)
graf2.update_yaxes(
    title_text = "trzna cena[miliard $]"    
)
graf2.update_xaxes(
    title_text = "profit[miliard $]"
)
graf2.show()

In [86]:
najboljsa_podjetja = podjetja[podjetja['profit_B'] > 10][podjetja['trzna_cena_B'] > 300]


graf10 = px.scatter(najboljsa_podjetja, x="profit_B", y="trzna_cena_B", text='ime')
graf10.update_layout(
    title = "Največja podjetja",
    title_font = {'size': 24},
    width = 700,
    height = 600
)
graf10.update_traces(
    textposition = 'top center'
)

graf10.update_yaxes(
    title_text = "trzna cena[miliard $]" ,  
)
graf10.update_xaxes(
    title_text = "profit[miliard $]",
    range = [0, 120]   
)
graf10.show()

Uspešnost podjetja bi se naivno dalo izračunati kot kvocient prodanega blaga in premoženske vrednosti. Količina profita na količino blaga pa naj predstavlja dodano vrednost ki jo podjetje prida svojemu izdelku. Spodji podatek kaže nepričakovana rezultata po katerih slednji kvaliteti nimata odvisnosti od mesta na lestvici. Podatek nemara ni tako presenetljiv, če upoštevamo, da se uspešnost najvidjeje prikazije pri manjših podjetjih.

In [87]:
graf6 = px.scatter(podjetja.drop([365]), x="Donosnost na premozensko vrednost", y="Donosnost na prodajo",
             color='mesto')
graf6.add_annotation(x = -0.023, y=-0.048,
    text="Vidimo, da nekatera podjetja beležijo izgubo.",
    showarrow=True,
    arrowhead=1,
    ax = 20,
    ay = -250,
    borderwidth=2,
    borderpad=4,
    bgcolor="#ADD8E6",
    opacity=0.8,
    arrowcolor="#636363",
    )
graf6.update_layout(
    title = "Prikaz nepovezanih spremenljivk",
    title_font = {'size': 24},
    width = 900,
    height = 450
)
graf6.update_yaxes(
    title_text = "Donosnost na kolicino prodanega"    
)
graf6.update_xaxes(
    title_text = "Donosnost glede na oceno premoženske vrednosti"
)
graf6.show()

Še zadnja tema v načrtu korelacija podatkov z letom_ustanovitve. Najbolj očitna sta primerjavi glede na panoge in celine ki dajeta pričakovane rezultate.

In [95]:
podjetja_po_panogi=podjetja_po_panogi.sort_values('leto_ustanovitve_median', ascending=False)

graf8 = go.Figure(data=[
    go.Bar(name='leto_ustanovitve', x=podjetja_po_panogi.index, y=podjetja_po_panogi.leto_ustanovitve_median),
])
graf8.update_layout(
    title = "Mediana leta ustanovitev",
    title_font = {'size': 24},
    width = 1200,
    height = 600,
)
graf8.update_xaxes(
        tickangle = 45,
        title_text = "Panoge",
        title_font = {"size": 12},
        title_standoff = 20)
graf8.update_yaxes(
    title_text = "Mediana leta ustanovitve",
    title_font = {"size" : 12},
    range = [1830,2020]
)


graf8.show()

In [94]:
podjetja_po_kontinentih = podjetja.groupby('kontinent').agg({'leto_ustanovitve' : ['mean', 'median']})
podjetja_po_kontinentih.columns = [x[0]+'_'+x[1] for x in podjetja_po_kontinentih.columns]
podjetja_po_kontinentih = podjetja_po_kontinentih.sort_values('leto_ustanovitve_mean', ascending=False)

graf9 = go.Figure(data=[
    go.Bar(name='povprečje', x=podjetja_po_kontinentih.index, y=podjetja_po_kontinentih.leto_ustanovitve_mean),
    go.Bar(name='mediana', x=podjetja_po_kontinentih.index, y=podjetja_po_kontinentih.leto_ustanovitve_median)
])
graf9.update_layout(
    title = "Čas ustanovitve podjetij po kontinentih",
    title_font = {'size': 24},
    legend = {
        'font' : {'size': 13},
    }
)
graf9.update_xaxes(
        tickangle = 45,
        title_text = "Kontinenti",
        title_font = {"size": 12},
        title_standoff = 20)
graf9.update_yaxes(
    title_text = "leto ustanovitve",
    title_font = {"size" : 12},
    range = [1900, 2020]
)

graf9.show()

Drugače leto ustanovitve na kaže posebnih odvisnosti od ostalih kvalitet. Za primer je spodnji graf, ki prikazuje njeno odvisnost od mesta na lestvici.

In [10]:
graf6 = px.scatter(podjetja, x="leto_ustanovitve", y="mesto")
graf6.update_layout(
    title = "Odvisnost leta ustanovitve od mesta na lestvici",
    title_font = {'size':24}
)
graf6.update_yaxes(
    title_text = "Mesto na lestvici"    
)
graf6.update_xaxes(
    title_text = "Leto ustanovitve"
)
graf6.show()